In [1]:
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
from PIL import Image
from sklearn.model_selection import train_test_split
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt


# Flash and PyTorch Lightning
from pl_flash.text import TextClassificationData, TextClassifier
import pytorch_lightning as pl

/home/teddy/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/home/teddy/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,
PyTorch version 1.6.0 available.
TensorFlow version 2.3.0 available.


In [2]:
data_path = Path("data/imdb")

if not data_path.exists():
    with urlopen("https://pl-flash-data.s3.amazonaws.com/imdb.zip") as resp:
        with ZipFile(BytesIO(resp.read())) as file:
            file.extractall("data/")

/home/teddy/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
pd.read_csv(data_path/"train.csv").head()

,review,sentiment
0,Japanese indie film with humor and philosophy ...,positive
1,Isaac Florentine has made some of the best wes...,negative
2,After seeing the low-budget shittier versions ...,negative
3,I've seen the original English version on vide...,positive
4,"Ahh, nuthin' like cheesy, explopitative, semi-...",negative


In [4]:
data = TextClassificationData.from_files(
    train_file=data_path/"train.csv",
    valid_file=data_path/"valid.csv",
    test_file=data_path/"test.csv",
    text_field="review",
    label_field="sentiment",
    batch_size=32
)

/home/teddy/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Checking /home/teddy/.cache/huggingface/datasets/d927c670fd53408efaea423294f823daf050357872041f191bfea8af06a952b6.03756fef6da334f50a7ff73608e21b5018229944ca250416ce7352e25d84a552.py for additional imports.
Found main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/csv/csv.py at /home/teddy/.cache/huggingface/modules/datasets_modules/datasets/csv
Found specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/csv/csv.py at /home/teddy/.cache/huggingface/modules/datasets_modules/datasets/csv/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd3

Done writing 22500 examples in 99118186 bytes /home/teddy/.cache/huggingface/datasets/csv/default-4c551d7a04ff9804/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277/tmpt2m67mqy.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Caching processed dataset at /home/teddy/.cache/huggingface/datasets/csv/default-4c551d7a04ff9804/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277/cache-16a4b77e3363d2e5.arrow


Done writing 2500 examples in 11005031 bytes /home/teddy/.cache/huggingface/datasets/csv/default-4c551d7a04ff9804/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277/tmpbqcwwuwh.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset
Caching processed dataset at /home/teddy/.cache/huggingface/datasets/csv/default-4c551d7a04ff9804/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277/cache-8b6ae889fbd14d76.arrow


Done writing 25000 examples in 110161107 bytes /home/teddy/.cache/huggingface/datasets/csv/default-4c551d7a04ff9804/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277/tmp87iwmjei.
Set __getitem__(key) output type to torch for ['input_ids', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for ['input_ids', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for ['input_ids', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


In [5]:
task = TextClassifier(num_classes=2, metrics=pl.metrics.Accuracy())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=5,
    log_every_n_steps=1,
)

trainer.fit(task, data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params
----------------------------------------------------------
0 | metrics | ModuleDict                    | 0     
1 | model   | BertForSequenceClassification | 109 M 


/home/teddy/anaconda3/lib/python3.7/site-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/teddy/anaconda3/lib/python3.7/site-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be 

/home/teddy/anaconda3/lib/python3.7/site-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/teddy/anaconda3/lib/python3.7/site-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be 